In [1]:
from li2021_odeint import change_params, create_fig3, lightsim_20min_5min, params
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from mxlpy import Simulator, make_protocol
from mxlpy_version import get_li2021
from tqdm import tqdm
from mxlbricks import get_li2021 as li2021_bricks

import mxlbricks.names as n

In [ ]:
fig3, axs = plt.subplot_mosaic([
        ["A"] * 7 + ["B"] * 7 + ["C"] * 7 + ["D"] * 7 + ["E"] * 7,
        ["F"] * 5 + ["G"] * 5 + ["H"] * 5 + ["I"] * 5 + ["J"] * 5 + ["K"] * 5 + ["L"] * 5
    ], figsize=(25,5))

light_100 = make_protocol([(20 * 60, {"PAR": 100}), (5 * 60, {"PAR": 0})])
light_500 = make_protocol([(20 * 60, {"PAR": 500}), (5 * 60, {"PAR": 0})])

g_types = ["WT", "clce2", "vccn1", "kea3", "clce2vccn1", "clce2kea3", "vccn1kea3", "vccn1clce2kea3"]

res_mxlpy = {}
for g_type in (pbar := tqdm(g_types)):
    pbar.set_description(f"Simulating '{g_type}' in mxlpy")
    m = get_li2021()
    
    if "clce2" in g_type:
        m.update_parameter("k_CLCE", 0)
    if "vccn1" in g_type:
        m.update_parameter("k_VCCN1", 0)
    if "kea3" in g_type:
        m.update_parameter("k_KEA", 0)
    
    res_mxlpy[g_type] = {}
    
    for par, prtc in zip(["100", "500"], [light_100, light_500]):
        s = Simulator(m)
        s.simulate_protocol(prtc, time_points_per_step=1000)
        variables, fluxes = s.get_result()
        res_mxlpy[g_type][par] = variables

res_mxlbricks = {}
for g_type in (pbar := tqdm(g_types)):
    pbar.set_description(f"Simulating '{g_type}' in mxlpy")
    m = get_li2021()
    
    if "clce2" in g_type:
        m.update_parameter("k_CLCE", 0)
    if "vccn1" in g_type:
        m.update_parameter("k_VCCN1", 0)
    if "kea3" in g_type:
        m.update_parameter("k_KEA", 0)
    
    res_mine[g_type] = {}
    
    for par, prtc in zip(["100", "500"], [light_100, light_500]):
        s = Simulator(m)
        s.simulate_protocol(prtc, time_points_per_step=1000)
        variables, fluxes = s.get_result()
        res_mine[g_type][par] = variables

for res, color in zip([res_mine["WT"]["100"], res_mine["WT"]["500"]], ["black", "red"]):
    axs["A"].plot(res.index / 60, 1-res["QA_red"], color=color, ls="dashed")
    axs["B"].plot(res.index / 60, res["Phi2"], color=color, ls="dashed")
    axs["C"].plot(res.index / 60, res["NPQ"], color=color, ls="dashed")

for ax_let, key in zip(list("FGHIJKL"), ["clce2", "vccn1", "kea3", "clce2vccn1", "clce2kea3", "vccn1kea3", "vccn1clce2kea3"]):
    ax = axs[ax_let]
    for par, color in zip(["100", "500"], ["black", "red"]):
        res = res_mine[key][par]
        ax.plot(res.index / 60, res["NPQ"] - res_mine["WT"][par]["NPQ"], color = color, ls="dashed")
        
x_coord = 0
width = 0.125
xcoords = []
for key in ["WT", "clce2", "vccn1", "kea3", "clce2vccn1", "clce2kea3", "vccn1kea3", "vccn1clce2kea3"]:
    axs["D"].scatter(x_coord - width, max(res_mine[key]["100"]["delta_pH_inVolts"]) - max(res_mine["WT"]["100"]["delta_pH_inVolts"]), color="black", marker="x")
    axs["D"].scatter(x_coord + width, max(res_mine[key]["500"]["delta_pH_inVolts"]) - max(res_mine["WT"]["500"]["delta_pH_inVolts"]), color="red", marker="x")
    
    max_Z = max(res_mine[key]["500"][n.zx()])
    max_V = max(res_mine[key]["500"][n.vx()])
    axs["E"].scatter(x_coord, max_Z / (max_Z + max_V), color="black", marker="x")
    xcoords += [x_coord]
    x_coord += 2
    
fig3.legend(
    [
        Line2D([0], [0], color="black", marker="s", markersize=7, ls="None"),
        Line2D([0], [0], color="red", marker="s", markersize=7, ls="None"),
        Line2D([0], [0], color="black", ls="dashed"),
        Line2D([0], [0], color="black", ls="solid")
    ],
    ["100 µmol photons m$^{-2}$ s$^{-1}$", "500 µmol photons m$^{-2}$ s$^{-1}$", "MxlPy", "Odeint"],
    ncols=4,
    frameon=False,
    loc="upper center",
)

plt.show()